In [1]:
from random import random, seed, randrange
from math import floor, pi
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
import pandas as pd
from scipy.integrate import simps, trapz, cumtrapz
from scipy.ndimage.filters import gaussian_filter1d
from time import time
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 9]

In [2]:
%run -i "../../carbon_walls/gcmc.py"

In [3]:
filename = "../data/" + "y0.75rep0" + ".csv"
data_dump = pd.read_csv(filename).copy()

for i in range(data_dump.shape[0]):
    if np.isnan( data_dump.yco_sim[i]):
        yco = data_dump.yco_res[i]
        P_res = data_dump.P_res[i]*10**5 #[Pa]
        T = data_dump.T_res[i] + 273.15 #K
        fco, fme = PR_Fugacity( P_res/10**5, T, yco )
        fco = fco*10**5
        fme = fme*10**5
        
        sf = False
        mega_verbose = False

        s_box = 57.15
        N_max = 50000
        Vol = s_box**3
        kb = 1.3806*10**(7) #[Pa*A^3/K]
        Nco = 0 #floor(fco*Vol/kb/T)
        Nme = 0 #floor(fme*Vol/kb/T)
        Nc = 0
        rc = s_box
        beta = 1/T
        zz_co = beta*fco
        zz_me = beta*fme
        delta = 1
        pi_move = 0.5

        N_moves = 100
        N_equil = int( np.round( 200000/N_moves) )
        N_prod = int( np.round( 100000/N_moves) )

        t0 = time()
        rhocov,rhomev,Env,Pv,Ncov, Nmev = mc_run()
        t = time()
        
        Nco = Ncov.mean()
        Nme = Nmev.mean()
        data_dump.yco_sim[i] = Nco/(Nco + Nme)
        data_dump.P_sim[i] = Pv.mean()*10 #[bar]
        data_dump.E_sim[i] = Env.mean()
        data_dump.rhoco[i] = rhocov.mean()
        data_dump.rhome[i] = rhomev.mean()
        data_dump.yco_s[i] = (rhocov/(rhocov + rhomev)).std()
        data_dump.P_s[i] = Pv.std()*10 #[bar]
        data_dump.rhoco_s[i] = rhocov.std()
        data_dump.rhome_s[i] = rhomev.std()
        data_dump.E_s[i] = Env.std()
        data_dump.time[i] = t - t0
        print( i, data_dump.yco_sim[i],  data_dump.P_sim[i], data_dump.rhoco[i], data_dump.rhome[i], data_dump.time[i] )
        #filename = "../data_output/yco" + str(yco) + "P" + str(P) + "/run" + str(i) + ".csv"
        data_dump.to_csv( filename, index = False)

0 0.7449325136848587 9.936868808691425 0.0001825294151004145 6.24987072425351e-05 32.77755904197693
1 0.7407367350180653 19.778377437578293 0.0003757253453826221 0.00013150661385517493 62.54672312736511
2 0.7382103868453197 29.55485320892035 0.0005807365172078886 0.00020594506782049002 94.80294108390808
3 0.7331652489335325 38.669798697454695 0.000785857890046359 0.000286012184523424 133.4034788608551
4 0.740438311813086 47.95375463607237 0.0010373618889445314 0.0003636486643916118 173.3535361289978
5 0.7314292889089365 55.271408313771595 0.00123773674288759 0.00045447980033274694 207.72492051124573
6 0.7305568847813728 69.78350332952884 0.0016698394694219805 0.000615868193057579 267.8149309158325
7 0.7229550954724104 74.94132310942524 0.0018772890998359684 0.0007193992859195336 315.7899680137634
8 0.714735854080188 82.51127111701003 0.0021276143711247493 0.0008491697918340698 363.25315976142883
9 0.7097155212214357 89.22929651534525 0.0026447388669163566 0.0010817385565514356 444.9114